In [1]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

## 2019 movies

In [ ]:
PATH = './SplitTrailers_crop/'
trailers = os.listdir(PATH)
trailers.sort()
print(len(trailers), trailers[:10])

## 2018 movies

In [6]:
PATH = './2018SplitTrailers_crop/'
df_onehot = pd.read_csv('./csv/filename2genreMat_2018.csv')
df_onehot

,id,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0328810,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,tt10005184,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt10017502,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,tt10043732,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,tt10048096,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,tt9866700,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1401,tt9879080,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1402,tt9891764,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1403,tt9904014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
trailers = df_onehot['id'].tolist()
trailers[:10]

['tt0328810',
 'tt10005184',
 'tt10017502',
 'tt10043732',
 'tt10048096',
 'tt10059624',
 'tt10077620',
 'tt10116528',
 'tt10160782',
 'tt10178206']

# A_AverageShotLength

In [8]:
A_li = []
for movie in trailers:
    frameCount = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' +scene + '/frames/'
        frameCount.append(len(os.listdir(framePath)))
    
#     print(frameCount)
    avg = np.mean(frameCount)
    std = np.std(frameCount)
    print(avg, std)
    
    A_li.append([avg, std])

Running: tt0328810
74.32432432432432 77.63255588122531
Running: tt10005184
128.11111111111111 168.8131409079574
Running: tt10017502
31.970588235294116 14.59751246761634
Running: tt10043732
41.670731707317074 28.35061614938854
Running: tt10048096
28.48 27.72885861336525
Running: tt10059624
69.64285714285714 93.34039635568692
Running: tt10077620
50.43589743589744 49.143325118288324
Running: tt10116528
132.35714285714286 99.4532245709058
Running: tt10160782
26.629032258064516 14.973346072667347
Running: tt10178206
100.26666666666667 76.16280807148154
Running: tt10182822
32.18260869565217 19.331246438005
Running: tt10186696
28.176470588235293 12.564145296054834
Running: tt10212512
30.671641791044777 20.38757893709948
Running: tt10233502
85.11111111111111 101.45710034234442
Running: tt10298956
69.42857142857143 69.71033361163104
Running: tt10322976
56.353846153846156 50.27506822677718
Running: tt10344086
75.0 52.40229002629561
Running: tt10379122
109.04761904761905 67.17743115240287
Running

In [9]:
A_arr = np.asarray(A_li)
np.save('./npy/2018A_arr.npy', A_arr)
len(A_arr)

1405

# B_ColorVariance
* https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_luv
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html

In [10]:
B_li = []
for movie in trailers: #order:
    rho = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        l = luv[:,:,0].ravel()
        u = luv[:,:,1].ravel()
        v = luv[:,:,2].ravel()
        x = np.asarray((l, u, v))
        cov = np.cov(x)
        det = np.linalg.det(cov)
        rho.append(det)
    
    avg = np.mean(rho)
    std = np.std(rho)
    print(avg, std)
    
    B_li.append([avg, std])

Running: tt0328810
7429596.013007359 6754839.757017302
Running: tt10005184
1056263.4396525347 2578889.3730854634
Running: tt10017502
3220427.9363187305 6070315.314554977
Running: tt10043732
5971662.823106107 17101600.83285153
Running: tt10048096
11678228.086752083 22263503.22996316
Running: tt10059624
31720023.7281803 40676136.5176146
Running: tt10077620
30357812.959164944 64114598.02058001
Running: tt10116528
2780284.691670873 4707617.573627054
Running: tt10160782
2243784.000355879 3978417.2039359314
Running: tt10178206
157144.88512865937 580153.1773766062
Running: tt10182822
37925569.8660351 279980446.10781837
Running: tt10186696
10631596.603357196 19801923.914988946
Running: tt10212512
4499704.621307288 22000003.46625037
Running: tt10233502
888790.918606402 1913762.1807559528
Running: tt10298956
1269934.320813021 2525434.571432871
Running: tt10322976
963044.238231098 2876127.3670830326
Running: tt10344086
11029683.94686463 14906021.95828414
Running: tt10379122
13511662.369765293 268

In [11]:
B_arr = np.asarray(B_li)
np.save('./npy/2018B_arr.npy', B_arr)
len(B_arr)

1405

# C_MotionContent
* https://blog.gtwang.org/programming/opencv-motion-detection-and-tracking-tutorial/

In [12]:
C_li = []
for movie in trailers: # [900:]:
    moving_li = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
    
        video  = PATH + movie + '/' + scene + '/clip.avi'

        cap = cv2.VideoCapture(video)

        w = int(cap.get(3))
        h = int(cap.get(4))

        # 計算畫面面積
        area = w * h

        # 初始化平均影像
        ret, frame = cap.read()

        avg = cv2.blur(frame,(4, 4))
        avg_float = np.float32(avg)
#         plt.imshow(frame)
#         plt.show()

        while(cap.isOpened()):
            # 讀取一幅影格
            ret, frame = cap.read()

            # 若讀取至影片結尾，則跳出
            if ret == False:
                break

            # 模糊處理
            blur = cv2.blur(frame, (4, 4))

            # 計算目前影格與平均影像的差異值
            diff = cv2.absdiff(frame, avg)

            # 將圖片轉為灰階
            gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # 篩選出變動程度大於門檻值的區域
            ret, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

            # 使用型態轉換函數去除雜訊
            kernel = np.ones((5, 5), np.uint8)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

            # 產生等高線
            cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            moving = 0
            for c in cnts:
                moving += cv2.contourArea(c)
                
            moving_li.append(moving/area)
            
#             for c in cnts:
#                 # 忽略太小的區域
#                 if cv2.contourArea(c) < 1000:
#                     continue


#             # 畫出等高線（除錯用）
#             cv2.drawContours(frame, cnts, -1, (0, 255, 255), 2)
#             plt.imshow(frame)
#             plt.show()

            # 更新平均影像
            cv2.accumulateWeighted(blur, avg_float, 0.01)
            avg = cv2.convertScaleAbs(avg_float)

        cap.release()
        cv2.destroyAllWindows()
    
    avg = np.mean(moving_li)
    std = np.std(moving_li)
    print(avg, std)
    
    C_li.append([avg, std])

Running: tt0328810
0.3364244009408313 0.33594326869652436
Running: tt10005184
0.09973185055368892 0.17080273013066433
Running: tt10017502
0.2683491626613395 0.3252183696876087
Running: tt10043732
0.31592309889581943 0.33043643883629714
Running: tt10048096
0.2964252601856748 0.31429400334713725
Running: tt10059624
0.29005881302209796 0.28828806803563617
Running: tt10077620
0.3501840260471298 0.2977930994863327
Running: tt10116528
0.20414772240195841 0.19442820906642425
Running: tt10160782
0.2236024705030461 0.24535867921597865
Running: tt10178206
0.05641373666065714 0.07546037068076807
Running: tt10182822
0.18020706533261893 0.28203530691125955
Running: tt10186696
0.2367593641451434 0.26955309181542236
Running: tt10212512
0.19686915028582955 0.24976609876863864
Running: tt10233502
0.09387688987706178 0.0949235008963231
Running: tt10298956
0.17153119070627826 0.27801983048696993
Running: tt10322976
0.2728566843319993 0.3057389990269048
Running: tt10344086
0.2267224927564283 0.28313909865

In [13]:
C_arr = np.asarray(C_li)
np.save('./npy/2018C_arr.npy', C_arr)
len(C_arr)

1405

# D_LightingKey

In [14]:
D_li = []
for movie in trailers:
    lk = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        value = hsv[:,:,2].ravel()
        mean = np.mean(value)
        std = np.std(value)
        lk.append(mean*std)

    avg = np.mean(lk)
    std = np.std(lk)
    print(avg, std)
    
    D_li.append([avg, std])

Running: tt0328810
8309.913009225189 4659.362101477487
Running: tt10005184
5180.699115383695 2028.1505587613976
Running: tt10017502
3663.7550480056293 2106.268326334429
Running: tt10043732
4184.885639883502 3340.76894051458
Running: tt10048096
4229.213104438426 2424.4736619376613
Running: tt10059624
7731.561261837901 2196.5649036779246
Running: tt10077620
6235.588053172122 3338.3171835721932
Running: tt10116528
3903.8223714004384 2354.606719077968
Running: tt10160782
1643.5244026477812 1164.242535836768
Running: tt10178206
1368.821693054663 1850.5367915345387
Running: tt10182822
4524.210956744031 3295.4449874107518
Running: tt10186696
3811.3283182845444 1955.8781498093574
Running: tt10212512
2643.426495496786 2708.8086218243475
Running: tt10233502
2970.7481977398324 2442.4916182669194
Running: tt10298956
3599.167911768439 2941.681891807719
Running: tt10322976
3151.2946152587474 2368.702245686453
Running: tt10344086
5134.244522536329 2607.378755873502
Running: tt10379122
5441.4995053499

In [15]:
D_arr = np.asarray(D_li)
np.save('./npy/2018D_arr.npy', D_arr)
len(D_arr)

1405

# Results

In [ ]:
# # 合併 2018 C_arr
# C_arr = np.append(np.load('./npy/2018C_arr_300.npy'), np.load('./npy/2018C_arr_600.npy'), axis=0)
# C_arr = np.append(C_arr, np.load('./npy/2018C_arr_900.npy'), axis=0)
# C_arr = np.append(C_arr, np.load('./npy/2018C_arr_900-end.npy'), axis=0)
# C_arr.shape

In [16]:
# reload computed values
A_arr = np.load('./npy/2018A_arr.npy')
B_arr = np.load('./npy/2018B_arr.npy')
C_arr = np.load('./npy/2018C_arr.npy')
D_arr = np.load('./npy/2018D_arr.npy')

In [17]:
print(A_arr.shape, B_arr.shape, C_arr.shape, D_arr.shape)

(1405, 2) (1405, 2) (1405, 2) (1405, 2)


In [25]:
# First time succeed!
results = pd.DataFrame({
    'Movie': trailers,
    'A_avg': A_arr[:,0],
    'A_std': A_arr[:,1],
    'B_avg': B_arr[:,0],
    'B_std': B_arr[:,1],
    'C_avg': C_arr[:,0],
    'C_std': C_arr[:,1],
    'D_avg' : D_arr[:,0],
    'D_std' : D_arr[:,1]
})
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std
0,tt0328810,74.324324,77.632556,7.429596e+06,6.754840e+06,0.336424,0.335943,8309.913009,4659.362101
1,tt10005184,128.111111,168.813141,1.056263e+06,2.578889e+06,0.099732,0.170803,5180.699115,2028.150559
2,tt10017502,31.970588,14.597512,3.220428e+06,6.070315e+06,0.268349,0.325218,3663.755048,2106.268326
3,tt10043732,41.670732,28.350616,5.971663e+06,1.710160e+07,0.315923,0.330436,4184.885640,3340.768941
4,tt10048096,28.480000,27.728859,1.167823e+07,2.226350e+07,0.296425,0.314294,4229.213104,2424.473662
...,...,...,...,...,...,...,...,...,...
1400,tt9866700,140.034483,81.652114,1.034090e+06,2.018781e+06,0.115961,0.170274,5452.573097,3761.139541
1401,tt9879080,83.636364,51.821850,1.771188e+06,2.408702e+06,0.058649,0.080815,1852.817936,1417.178866
1402,tt9891764,32.716981,19.979119,7.569638e+06,1.308621e+07,0.243142,0.287351,4391.310240,2942.670602
1403,tt9904014,108.547619,73.594838,3.773972e+05,6.156485e+05,0.339983,0.243896,2960.448579,2322.644041


In [26]:
results.to_csv('./csv/computed_2018.csv', index = 0)

## Normalize

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
def normalizing(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    print('Data range:', scaler.data_range_)
    return scaler.transform(data)

In [21]:
cols = results.columns[1:].tolist()
cols

['A_avg', 'A_std', 'B_avg', 'B_std', 'C_avg', 'C_std', 'D_avg', 'D_std']

In [22]:
for col in cols:
    arr = np.asarray(results[col].tolist()).reshape(-1, 1)
    results[col+'_norm'] = normalizing(arr)

Data range: [7069.26666667]
Data range: [3254.46670832]
Data range: [1.17151282e+09]
Data range: [7.22238896e+09]
Data range: [0.75492767]
Data range: [0.4034122]
Data range: [15162.63539501]
Data range: [7670.52994464]


In [23]:
results.to_csv('./csv/computed_2018.csv', index = 0)
results

,Movie,A_avg,A_std,B_avg,B_std,C_avg,C_std,D_avg,D_std,A_avg_norm,A_std_norm,B_avg_norm,B_std_norm,C_avg_norm,C_std_norm,D_avg_norm,D_std_norm
0,tt0328810,74.324324,77.632556,7.429596e+06,6.754840e+06,0.336424,0.335943,8309.913009,4659.362101,0.007298,0.023854,0.006342,0.000935,0.445638,0.832754,0.533451,0.607437
1,tt10005184,128.111111,168.813141,1.056263e+06,2.578889e+06,0.099732,0.170803,5180.699115,2028.150559,0.014906,0.051871,0.000902,0.000357,0.132108,0.423395,0.327074,0.264408
2,tt10017502,31.970588,14.597512,3.220428e+06,6.070315e+06,0.268349,0.325218,3663.755048,2106.268326,0.001307,0.004485,0.002749,0.000840,0.355463,0.806169,0.227029,0.274592
3,tt10043732,41.670732,28.350616,5.971663e+06,1.710160e+07,0.315923,0.330436,4184.885640,3340.768941,0.002679,0.008711,0.005097,0.002368,0.418481,0.819104,0.261399,0.435533
4,tt10048096,28.480000,27.728859,1.167823e+07,2.226350e+07,0.296425,0.314294,4229.213104,2424.473662,0.000813,0.008520,0.009969,0.003083,0.392654,0.779089,0.264322,0.316076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,tt9866700,140.034483,81.652114,1.034090e+06,2.018781e+06,0.115961,0.170274,5452.573097,3761.139541,0.016593,0.025089,0.000883,0.000280,0.153606,0.422083,0.345005,0.490336
1401,tt9879080,83.636364,51.821850,1.771188e+06,2.408702e+06,0.058649,0.080815,1852.817936,1417.178866,0.008615,0.015923,0.001512,0.000334,0.077689,0.200329,0.107595,0.184756
1402,tt9891764,32.716981,19.979119,7.569638e+06,1.308621e+07,0.243142,0.287351,4391.310240,2942.670602,0.001412,0.006139,0.006461,0.001812,0.322074,0.712302,0.275013,0.383633
1403,tt9904014,108.547619,73.594838,3.773972e+05,6.156485e+05,0.339983,0.243896,2960.448579,2322.644041,0.012139,0.022613,0.000322,0.000085,0.450352,0.604582,0.180645,0.302801
